In [1]:
# --- Cell 1: Imports, config, GUI probe ---

import sys, time, datetime
from pathlib import Path

import numpy as np
import cv2
import mediapipe as mp
import depthai as dai

# Preview / record settings
PREVIEW_W, PREVIEW_H, FPS = 960, 540, 30
WINDOW_NAME = "OAK-D Pose (cam0 | cam1)"
FONT = cv2.FONT_HERSHEY_SIMPLEX
# Fast/low-res mode: try to pull a small preview from the device (highest FPS),
# otherwise fall back to full-res and downscale on host.
FAST_LOWRES = True          # toggle this to False to go back to full-res
LOWRES_W, LOWRES_H = 320, 180   # tiny, very fast for MediaPipe
TARGET_FPS = 60                 # request high fps if the node allows it

# If fast mode is on, also shrink preview/record sizes to match for max throughput
if FAST_LOWRES:
    PREVIEW_W, PREVIEW_H = LOWRES_W, LOWRES_H


def can_open_cv_window(force_headless: bool = False) -> bool:
    if force_headless:
        return False
    try:
        cv2.namedWindow("_probe_", cv2.WINDOW_NORMAL)
        cv2.imshow("_probe_", np.zeros((1, 1, 3), dtype="uint8"))
        cv2.waitKey(1)
        cv2.destroyWindow("_probe_")
        return True
    except Exception:
        return False

HAS_GUI = can_open_cv_window(False)
print(f"DepthAI: {dai.__version__} | OpenCV GUI available: {HAS_GUI}")


DepthAI: 3.1.0 | OpenCV GUI available: True


objc[56213]: Class CVWindow is implemented in both /Users/manolisfragkidakis/miniconda3/envs/cameras/lib/python3.11/site-packages/cv2/cv2.abi3.so (0x17459a648) and /Users/manolisfragkidakis/miniconda3/envs/cameras/lib/python3.11/site-packages/palace.dylibs/libdepthai-core.dylib (0x31b281368). This may cause spurious casting failures and mysterious crashes. One of the duplicates must be removed or renamed.
objc[56213]: Class CVView is implemented in both /Users/manolisfragkidakis/miniconda3/envs/cameras/lib/python3.11/site-packages/cv2/cv2.abi3.so (0x17459a670) and /Users/manolisfragkidakis/miniconda3/envs/cameras/lib/python3.11/site-packages/palace.dylibs/libdepthai-core.dylib (0x31b281390). This may cause spurious casting failures and mysterious crashes. One of the duplicates must be removed or renamed.
objc[56213]: Class CVSlider is implemented in both /Users/manolisfragkidakis/miniconda3/envs/cameras/lib/python3.11/site-packages/cv2/cv2.abi3.so (0x17459a698) and /Users/manolisfragki

In [2]:
# --- Cell 2: Device discovery helpers (tolerant) ---

def list_devices():
    return dai.Device.getAllAvailableDevices()

def pick_devices(required=2, allow_less=True):
    devs = list_devices()
    if len(devs) >= required:
        return devs[:required]
    if allow_less and len(devs) > 0:
        print(f"[WARN] Expected {required} devices, found {len(devs)}. Running with {len(devs)}.")
        return devs
    raise RuntimeError(f"Need {required} OAK devices, found {len(devs)}.")

def pick_rgb_socket(device: dai.Device):
    sockets = list(device.getConnectedCameras())
    if not sockets:
        raise RuntimeError("No cameras connected/reported on this device.")
    for s in sockets:
        if str(s).endswith("RGB") or s == getattr(dai.CameraBoardSocket, "RGB", None):
            return s
    return sockets[0]


In [3]:
# --- Cell 3: MediaPipe Pose wrapper ---

class PoseEstimator:
    def __init__(self, model_complexity=1):
        mp_pose = mp.solutions.pose
        self.pose = mp_pose.Pose(
            static_image_mode=False,
            model_complexity=model_complexity,
            enable_segmentation=False,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5,
        )
        self.drawer = mp.solutions.drawing_utils
        self.styles = mp.solutions.drawing_styles
        self.connections = mp_pose.POSE_CONNECTIONS

    def process_and_draw(self, frame_bgr):
        # frame_bgr is modified in-place (drawing)
        res = self.pose.process(cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB))
        if res.pose_landmarks:
            self.drawer.draw_landmarks(
                frame_bgr, res.pose_landmarks, self.connections,
                landmark_drawing_spec=self.styles.get_default_pose_landmarks_style()
            )
        return frame_bgr

    def close(self):
        self.pose.close()


In [4]:
# --- Cell 4: Recording & playback utilities ---

class DualRecorder:
    def __init__(self, out_dir, fps=FPS, size=(PREVIEW_W, PREVIEW_H)):
        out_dir = Path(out_dir)
        out_dir.mkdir(parents=True, exist_ok=True)
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.f0 = str(out_dir / f"cam0_{ts}.mp4")
        self.f1 = str(out_dir / f"cam1_{ts}.mp4")
        self.w0 = cv2.VideoWriter(self.f0, fourcc, fps, size)
        self.w1 = cv2.VideoWriter(self.f1, fourcc, fps, size)
        self.active = True
        print(f"[REC] → {self.f0}\n[REC] → {self.f1}")

    def write(self, f0, f1):
        if not self.active:
            return
        if f0 is not None: self.w0.write(f0)
        if f1 is not None: self.w1.write(f1)

    def stop(self):
        if self.active:
            self.w0.release(); self.w1.release()
            self.active = False
            print("[REC] stopped")

class DualPlayer:
    def __init__(self, f0, f1):
        self.cap0, self.cap1 = cv2.VideoCapture(f0), cv2.VideoCapture(f1)
        if not self.cap0.isOpened() or not self.cap1.isOpened():
            raise RuntimeError("Playback files not found or unreadable.")
        print(f"[PLAY] {f0}\n[PLAY] {f1}")

    def read(self):
        ok0, f0 = self.cap0.read()
        ok1, f1 = self.cap1.read()
        if not ok0 and not ok1:
            return None, None, False
        return (f0 if ok0 else None), (f1 if ok1 else None), True

    def stop(self):
        self.cap0.release(); self.cap1.release()

def latest_pair(base="recordings"):
    base = Path(base)
    if not base.exists(): return None, None
    dirs = sorted([p for p in base.iterdir() if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
    for d in dirs:
        c0 = max(d.glob("cam0_*.mp4"), default=None, key=lambda p: p.stat().st_mtime if p.exists() else 0)
        c1 = max(d.glob("cam1_*.mp4"), default=None, key=lambda p: p.stat().st_mtime if p.exists() else 0)
        if c0 and c1: return str(c0), str(c1)
    return None, None


In [5]:
# --- Cell 5: Build per-device pipelines using the new API style (fast/low-res aware) ---

def _request_preview_or_full(builder, width, height):
    """
    Try to get a low-res preview output from the on-device scaler.
    Fallback to full resolution if preview isn't available in this build.
    Returns (queue, using_preview: bool)
    """
    # Prefer preview (low-res)
    if FAST_LOWRES and hasattr(builder, "requestPreviewOutput"):
        try:
            # Some builds take (w,h), some use named args — try both
            try:
                qbuilder = builder.requestPreviewOutput(width, height)
            except TypeError:
                qbuilder = builder.requestPreviewOutput(width=width, height=height)
            q = qbuilder.createOutputQueue()
            return q, True
        except Exception:
            pass

    # Fallback: full resolution
    q = builder.requestFullResolutionOutput().createOutputQueue()
    return q, False


def make_output_queue_for_device(device: dai.Device):
    """
    Using the 'new' API:
      with dai.Pipeline(device) as pipeline:
          cam = pipeline.create(dai.node.Camera).build(socket)
          (preview or full) -> OutputQueue
    Returns (pipeline_context_manager, socket_name).
    """
    socket = pick_rgb_socket(device)
    socket_name = str(socket)

    class _PipeWrapper:
        def __enter__(self):
            self.cm = dai.Pipeline(device)
            self.pipeline = self.cm.__enter__()

            # Build camera node for this socket
            cam_builder = self.pipeline.create(dai.node.Camera).build(socket)

            # If the builder exposes FPS control in this API, request high FPS
            for attr in ("setFps", "setFrameRate"):
                if hasattr(cam_builder, attr):
                    try:
                        getattr(cam_builder, attr)(TARGET_FPS)
                    except Exception:
                        pass

            # Prefer preview output (low-res) if available
            self.queue, self.using_preview = _request_preview_or_full(
                cam_builder, LOWRES_W, LOWRES_H
            )

            self.pipeline.start()
            return self

        def __exit__(self, exc_type, exc, tb):
            try:
                if hasattr(self.pipeline, "stop"):
                    self.pipeline.stop()
            except Exception:
                pass
            return self.cm.__exit__(exc_type, exc, tb)

        def isRunning(self):
            return self.pipeline.isRunning()

        def get(self):
            return self.queue.get()

    return _PipeWrapper(), socket_name


In [6]:
# --- Device probe ---
import depthai as dai

devs = dai.Device.getAllAvailableDevices()
print(f"Detected devices: {len(devs)}")
for i, d in enumerate(devs):
    mx = getattr(d, "mxid", None)
    if mx is None and hasattr(d, "getMxId"):
        try: mx = d.getMxId()
        except: mx = None
    print(f"[{i}] mxid={mx or '<unknown>'} | attrs={sorted(set([k for k in dir(d) if not k.startswith('_')]))}")


Detected devices: 2
[0] mxid=<unknown> | attrs=['deviceId', 'getDeviceId', 'getXLinkDeviceDesc', 'name', 'platform', 'protocol', 'state', 'status']
[1] mxid=<unknown> | attrs=['deviceId', 'getDeviceId', 'getXLinkDeviceDesc', 'name', 'platform', 'protocol', 'state', 'status']


In [ ]:
# --- Cell 6: Main loop (works with 1 or 2 devices) ---

def main(out_dir="recordings", auto_record=False):
    dev_infos = pick_devices(required=2, allow_less=True)

    # Open devices we actually have
    devices = [dai.Device(di) for di in dev_infos]

    # Build per-device pipeline/queue wrappers
    pipes = []
    names = []
    for dev in devices:
        p, name = make_output_queue_for_device(dev)
        pipes.append(p)
        names.append(name)

    # Pose estimators (one per stream we have)
    poses = [PoseEstimator() for _ in pipes]

    # Session dir
    session_dir = Path(out_dir) / datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    session_dir.mkdir(parents=True, exist_ok=True)

    recorder = None
    player = None
    live = True

    if HAS_GUI:
        cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)
        cv2.resizeWindow(WINDOW_NAME, PREVIEW_W * 2, PREVIEW_H)

    if auto_record:
        recorder = DualRecorder(session_dir, fps=FPS, size=(PREVIEW_W, PREVIEW_H))

    t_prev = time.time()

    try:
        # Enter all available pipelines (1 or 2)
        ctxs = [p.__enter__() for p in pipes]
        try:
            # Helper to check running state
            def all_running():
                return all(p.isRunning() for p in ctxs)

            while all_running():
                if live:
                    # Read a frame per available pipe
                    frames = [ctx.get().getCvFrame() for ctx in ctxs]
                else:
                    f0, f1, ok = player.read()
                    if not ok:
                        print("[PLAY] done.")
                        if player: player.stop(); player = None
                        live = True
                        continue
                    frames = [f0, f1]

                # Process frames we have
                processed = []
                for f, pose in zip(frames, poses):
                    if f is None:
                        processed.append(None)
                        continue
                    if (f.shape[1], f.shape[0]) != (PREVIEW_W, PREVIEW_H):
                        f = cv2.resize(f, (PREVIEW_W, PREVIEW_H))
                    processed.append(pose.process_and_draw(f))

                # If only one device, synthesize the second pane as black
                if len(processed) == 1:
                    if processed[0] is None:
                        left = np.zeros((PREVIEW_H, PREVIEW_W, 3), np.uint8)
                    else:
                        left = processed[0]
                    right = np.zeros_like(left)
                else:
                    left = processed[0] if processed[0] is not None else np.zeros((PREVIEW_H, PREVIEW_W, 3), np.uint8)
                    right = processed[1] if processed[1] is not None else np.zeros((PREVIEW_H, PREVIEW_W, 3), np.uint8)

                combined = cv2.hconcat([left, right])

                # FPS & status
                now = time.time()
                fps = 1.0 / max(1e-6, now - t_prev)
                t_prev = now
                label = ("LIVE" if live else "PLAY") + (" + REC" if (recorder and recorder.active) else "")
                cv2.putText(combined, f"{label}  FPS: {fps:.1f}", (10, 30), FONT, 0.9, (0, 255, 0), 2)

                # Show
                key = 255
                if HAS_GUI:
                    cv2.imshow(WINDOW_NAME, combined)
                    key = cv2.waitKey(1) & 0xFF

                # Record (always write two panes for consistency)
                if recorder and recorder.active:
                    # If we only have one real camera, the second file will be black video.
                    recorder.write(left, right)

                # Hotkeys
                if key == ord('q'):
                    break
                elif key == ord('r') and HAS_GUI:
                    if recorder and recorder.active:
                        recorder.stop(); recorder = None
                    else:
                        recorder = DualRecorder(session_dir, fps=FPS, size=(PREVIEW_W, PREVIEW_H))
                elif key == ord('p') and HAS_GUI:
                    if live:
                        f0path, f1path = latest_pair(out_dir)
                        if not f0path or not f1path:
                            print("[PLAY] No recordings found.")
                        else:
                            player = DualPlayer(f0path, f1path)
                            live = False
                    else:
                        if player: player.stop(); player = None
                        live = True
                elif key == ord('s') and HAS_GUI:
                    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
                    cv2.imwrite(str(session_dir / f"cam0_snap_{ts}.png"), left)
                    cv2.imwrite(str(session_dir / f"cam1_snap_{ts}.png"), right)
                    print("[SNAP] saved")

        finally:
            # Exit all pipeline contexts
            for p in pipes[::-1]:
                try: p.__exit__(None, None, None)
                except: pass
    finally:
        # Cleanup
        if recorder and recorder.active:
            recorder.stop()
        if player:
            player.stop()
        for dev in devices:
            try: dev.close()
            except: pass
        for pose in poses:
            try: pose.close()
            except: pass
        if HAS_GUI:
            try: cv2.destroyAllWindows()
            except: pass


: 

In [ ]:

main(out_dir="recordings", auto_record=False)


/var/folders/n8/ty4phdkj5814s93r8dpd5vqm0000gn/T/ipykernel_56213/2833358920.py:20: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  if str(s).endswith("RGB") or s == getattr(dai.CameraBoardSocket, "RGB", None):
I0000 00:00:1763039863.702212 5936360 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1 Pro
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1763039863.709928 5936360 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1 Pro
W0000 00:00:1763039863.781230 5936874 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1763039863.787733 5936887 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1763039863.794314 5936874 inference_feedback_manager.cc:114] Feedback manager requires a

[2025-11-13 14:18:04.644] [depthai] [error] Communication exception - possible device error/misconfiguration. Original message 'Couldn't read data from stream: '__x_0_0' (X_LINK_ERROR)'
[14442C10B11ED7D600] [1.1.3] [1763039885.750] [host] [warning] Closed connection

[14442C10B11ED7D600] [1.1.3] [1763039885.750] [host] [warning] Attempting to reconnect. Timeout is 10000ms

[14442C10B11ED7D600] [1.1.3] [1763039890.975] [host] [warning] Reconnection successful

[2025-11-13 14:18:20.518] [depthai] [error] Communication exception - possible device error/misconfiguration. Original message 'Couldn't read data from stream: '__x_0_0' (X_LINK_ERROR)'
[14442C10B11ED7D600] [1.1.3] [1763039901.500] [host] [warning] Closed connection

[14442C10B11ED7D600] [1.1.3] [1763039901.500] [host] [warning] Attempting to reconnect. Timeout is 10000ms

[14442C10B11ED7D600] [1.1.3] [1763039908.384] [host] [warning] Reconnection successful

